# GONZAG cloud example

In [1]:
import sys
from os import getenv
import warnings
warnings.filterwarnings("ignore")

In [2]:
from gonzag.config import *
from gonzag.io import *
from gonzag.utils import *
from gonzag.mod2sat import *
from gonzag.bilin_mapping import *
from gonzag.spectralysis import *


## Data

In [3]:
import xarray as xr
osn_url = 'https://ncsa.osn.xsede.org/Pangeo/pangeo-forge/swot_adac/eNATL60_surface_region_1'
dsmod = xr.open_zarr(osn_url, consolidated=True)

In [4]:
dsmod

<xarray.Dataset>
Dimensions:        (time_counter: 2136, x: 611, y: 763)
Coordinates:
    depth          (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1f            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1t            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1u            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1v            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e2f            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e2t            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e2u            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e2v            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    lat            (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    lon            (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    nav_lat        (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    nav_lon        (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    time_centered  (time_counter) datetime64[ns] dask.array<chunksize=(72,), meta=np.ndarray>
  * time_counter   (time_counter) datetime64[ns] 2010-02-01T00:30:00 ... 2010...
Dimensions without coordinates: x, y
Data variables:
    fmask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>
    qt_oce         (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    somecrty       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sometauy       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sosaline       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sossheig       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sosstsst       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sowaflup       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sozocrtx       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sozotaux       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    tmask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>
    umask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>
    vmask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>

In [5]:
name_ssh_mod = 'sossheig'
name_lsm_mod = 'tmask' 
l_griddist = False

In [6]:
from intake import open_catalog
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/altimetry.yaml")
dssat  = cat["al"].to_dask()

In [7]:
dssat

<xarray.Dataset>
Dimensions:         (time: 34340107)
Coordinates:
    latitude        (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    longitude       (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
  * time            (time) datetime64[ns] 2013-03-14T05:44:50.445185024 ... 2...
Data variables:
    cycle           (time) int16 dask.array<chunksize=(34340107,), meta=np.ndarray>
    dac             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    lwe             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    mdt             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    ocean_tide      (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    sla_filtered    (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    sla_unfiltered  (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    track           (time) int16 dask.array<chunksize=(34340107,), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.6
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    cdm_data_type:             Swath
    comment:                   Sea surface height measured by altimeters refe...
    contact:                   servicedesk.cmems@mercator-ocean.eu
    creator_email:             servicedesk.cmems@mercator-ocean.eu
    creator_name:              CMEMS - Sea Level Thematic Assembly Center
    creator_url:               http://marine.copernicus.eu
    history:                   2019-02-20T19:39:52Z: Creation
    institution:               CLS, CNES
    keywords:                  Oceans > Ocean Topography > Sea Surface Height
    keywords_vocabulary:       NetCDF COARDS Climate and Forecast Standard Names
    license:                   http://marine.copernicus.eu/web/27-service-com...
    platform:                  Altika
    processing_level:          L3
    product_version:           2019
    project:                   COPERNICUS MARINE ENVIRONMENT MONITORING SERVI...
    references:                http://marine.copernicus.eu
    software_version:          6.2_DUACS_DT2018_baseline
    source:                    Altika measurements
    ssalto_duacs_comment:      The reference mission used for the altimeter i...
    standard_name_vocabulary:  NetCDF Climate and Forecast (CF) Metadata Conv...
    summary:                   SSALTO/DUACS Delayed-Time Level-3 sea surface ...
    title:                     DT Altika Global Ocean Along track SSALTO/DUAC...

In [8]:
name_ssh_sat='sla_unfiltered'

## Time overlap between model and satellite ?
Assume that model is less than 1 year, satellite has at least one whole year, we take the overlapping period without considering which year is this

In [9]:
    (date1,date2), (Nts,Ntm) = GetTimeOverlap( dssat , dsmod )
    print(' *** Time overlap between model and satellite in UNIX epoch time: it1, it2',date1,'--',date2)


 *** [GetTimeInfo()] Getting calendar/time info in dataset ...
   => time/record dimension is "time"
   => first and last date:  2013-03-14T05:44:50.445185024 -- 2015-03-31T22:41:36.909480960
 *** [GetTimeInfo()] Getting calendar/time info in dataset ...
   => time/record dimension is "time_counter"
   => first and last date:  2010-02-01T00:30:00.000000000 -- 2010-04-30T23:30:00.000000000

 *** Earliest/latest dates:
   => for satellite data: 2013-03-14T05:44:50.445185024 2015-03-31T22:41:36.909480960 
   => for model     data: 2010-02-01T00:30:00.000000000 2010-04-30T23:30:00.000000000 

 *** Time overlap between model and satellite in UNIX epoch time: it1, it2 2014-02-01 -- 2014-04-30


## Create object `ModelGrid` containing all the model (aka _source_) 2D+T domain grid info

In [10]:
  
    ModelGrid = ModGrid( dsmod, date1 , date2 , dsmod, name_lsm_mod, distorded_grid=False )


 *** [GetModelCoor()] Read model latitude (variable is "lat", with 2 dimensions! (763, 611) 

 *** [GetModelCoor()] Read model longitude (variable is "lon", with 2 dimensions! (763, 611) 


 *** what we use to define model land-sea mask:
    => "tmask" in dataset 

 *** [GridResolution()] Based on the longitude array, the model resolution ~=  0.016657295  degrees 

 *** Skipping computation of angle distortion of the model grid! ("-D" option not invoked)...

 *** About model gridded (source) domain:
     * shape =  (763, 611)
     * horizontal resolution:  0.016657295  degrees or  1.8507920147292316  km
     * Is this a global domain w.r.t longitude:  False
       ==> this is a regional domain
       ==> working in the [0:360] frame...
     * lon_min, lon_max =  281.99 292.17
     * lat_min, lat_max =  30.01 40.01
     * should we pay attention to possible STRONG local distorsion in the grid:  False
     * number of time records of interest for the interpolation to come:  2136
       =

In [11]:
SatelliteTrack = SatTrack( dssat, date1, date2, Np=Nts, \
                              domain_bounds=ModelGrid.domain_bounds, l_0_360=ModelGrid.l360 )

 *** [SatTrack()] Analyzing the time vector in dataset ...
 *** [GetSatCoor()] reading "latitude" in dataset ...
   => 4150072 records read...

 *** [GetSatCoor()] reading "longitude" in dataset ...
   => 4150072 records read...


 *** About satellite track (target) domain:
     * number of time records of interest for the interpolation to come:  10236
       ==> time record indices: 14883510 to 19033581, included



In [12]:
(Nj,Ni) = ModelGrid.shape
d_found_km = rfactor*ModelGrid.HResDeg*deg2km

In [13]:
np_box_radius = SearchBoxSize( ModelGrid.HResDeg*deg2km, search_box_w_km )

In [26]:
Yt=SatelliteTrack.lat
Xt=SatelliteTrack.lon
Ys=ModelGrid.lat
Xs=ModelGrid.lon
src_grid_local_angle=ModelGrid.xangle
k_ew_per=ModelGrid.EWPer
rd_found_km=d_found_km
np_box_r=np_box_radius
rfound = rd_found_km


In [17]:
Nt = SatelliteTrack.size
if_talk = Nt//nb_talk

In [18]:
freq_talk=if_talk

In [20]:
NP = nmp.zeros((Nt,2)  , dtype=nmp.int64)

In [21]:
SM = nmp.zeros((Nt,4,2), dtype=nmp.int64) 

In [22]:
WB = nmp.zeros((Nt,4))

In [23]:
ftalk = int(freq_talk)

In [24]:
nprad  = np_box_r
xnp = nmp.zeros((Nt,2), dtype=nmp.int64)
[jj,ji] = [nprad,nprad] ; # stupid first guess here...


In [27]:
jt=0
[jj,ji] = NearestPoint( (Yt[jt],Xt[jt]), Ys, Xs, \
                                    rd_found_km=rfound, j_prv=jj, i_prv=ji, np_box_r=nprad )

AttributeError: 'Array' object has no attribute 'item'

In [29]:
pcoor_trg=(Yt[jt],Xt[jt])
rd_found_km=rfound
j_prv=jj
i_prv=ji
np_box_r=nprad

In [ ]:
round(Ys[jj,ji],3)

In [30]:
print(Yt[jt],Xt[jt])

39.973308 289.681787


In [31]:
(yT,xT) = pcoor_trg
(Ny,Nx) = Ys.shape

In [32]:
    jy, jx = -1,-1 ; # Flag value...
    j1=max(j_prv-np_box_r,0) ; j2=min(j_prv+np_box_r+1,Ny)
    i1=max(i_prv-np_box_r,0) ; i2=min(i_prv+np_box_r+1,Nx)
    lfound = False
    rfnd = rd_found_km
    igo = 0


In [44]:
        xd = Haversine( yT, xT,  Ys[j1:j2,i1:i2], Xs[j1:j2,i1:i2] )
        jy, jx = find_j_i_min( xd.values )


In [45]:
jy,jx

(270, 270)

In [42]:
k = xd.values.argmin()

In [43]:
k

73440

In [37]:
xd

<xarray.DataArray (y: 271, x: 271)>
dask.array<mul, shape=(271, 271), dtype=float32, chunksize=(271, 271), chunktype=numpy.ndarray>
Coordinates:
    depth    (y, x) float32 dask.array<chunksize=(271, 271), meta=np.ndarray>
    e1f      (y, x) float64 dask.array<chunksize=(271, 271), meta=np.ndarray>
    e1t      (y, x) float64 dask.array<chunksize=(271, 271), meta=np.ndarray>
    e1u      (y, x) float64 dask.array<chunksize=(271, 271), meta=np.ndarray>
    e1v      (y, x) float64 dask.array<chunksize=(271, 271), meta=np.ndarray>
    e2f      (y, x) float64 dask.array<chunksize=(271, 271), meta=np.ndarray>
    e2t      (y, x) float64 dask.array<chunksize=(271, 271), meta=np.ndarray>
    e2u      (y, x) float64 dask.array<chunksize=(271, 271), meta=np.ndarray>
    e2v      (y, x) float64 dask.array<chunksize=(271, 271), meta=np.ndarray>
    lat      (y, x) float32 dask.array<chunksize=(271, 271), meta=np.ndarray>
    lon      (y, x) float32 dask.array<chunksize=(271, 271), meta=np.ndarray>
    nav_lat  (y, x) float32 dask.array<chunksize=(271, 271), meta=np.ndarray>
    nav_lon  (y, x) float32 dask.array<chunksize=(271, 271), meta=np.ndarray>
Dimensions without coordinates: y, x

In [46]:
        lfound = ( xd[jy,jx] < rfnd )


In [33]:
    while (not lfound) and igo<5 :
        if igo>1 and not lfound:
            rfnd = 1.25*rfnd ; # increasing validation distance criterion by 25 %


AttributeError: 'Array' object has no attribute 'item'

In [ ]:
for jt in range(Nt):
    ltalk = ((jt+1)%ftalk==0)
    if ltalk:
        print('      +++ Treated point: '+str(jt+1)+'/'+str(Nt)+'\n          ==> Sat. coordinates:    ', round(Yt[jt],3), round(Xt[jt],3))
        [jj,ji] = NearestPoint( (Yt[jt],Xt[jt]), Ys, Xs, \
                                    rd_found_km=rfound, j_prv=jj, i_prv=ji, np_box_r=nprad )

            # Exclude points at first and last column if no E-W periodicity:
            if ji==0 or ji==self.Ni-1 and self.kewp == -1: [jj,ji] = [-1,-1]
            # Exclude points at first and last row:
            if jj==0 or jj==self.Nj-1:                     [jj,ji] = [-1,-1]
            
            xnp[jt,:] = [jj,ji]

            if ltalk:
                print('          ==> Model nearest point: ', \
                      round(self.Ys[jj,ji],3),round(self.Xs[jj,ji]%360.,3),' (',jj,ji,')')